In [16]:
#!pip install kafka-python
#!pip install faker
# currently using kafka-python-2.0.2, Faker-28.0.0
from kafka import KafkaProducer
from kafka import KafkaConsumer
import sys
import time
import random
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import datetime as tm
from faker import Faker

In [17]:
#Kafka source details - this is for EDF Kafka Wire Protocol - so the broker is where Data Access Gateway is running and its port
server = "13.215.254.242:9092"
topic = "freshtopic"
print("BROKER: " + server)
print("TOPIC: " + topic)

BROKER: 13.215.254.242:9092
TOPIC: freshtopic


In [18]:
# this API call will change based on the actual Kafka broker 
#PRODUCER
print("\n**Starting Producer**")
producer=KafkaProducer(bootstrap_servers=[server],
#                        security_protocol='SASL_PLAINTEXT',
#                        sasl_mechanism='PLAIN',
#                        sasl_plain_username='mapr',
#                        sasl_plain_password='mapr'
                      )

<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: connecting to 13.215.254.242:9092 [('13.215.254.242', 9092) IPv4]
Probing node bootstrap-0 broker version
<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: Connection complete.
Broker version identified as 2.5.0
Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
<BrokerConnection node_id=0 host=13.215.254.242:9092 <connected> [IPv4 ('13.215.254.242', 9092)]>: Closing connection. 



**Starting Producer**


In [19]:
numMsgProduced = 0
fake = Faker()
for h in range(100):
    record = {
        "id": h+random.randint(1,100000),
        "name": fake.name(),
        "address": fake.address(),
        "amount": random.randint(1,1000000)
    }
    print(record)
    producer.send(topic, json.dumps(record).encode('utf-8'))
    numMsgProduced += 1
producer.flush()
print("Messages produced: " + str(numMsgProduced))
time.sleep(2)

<BrokerConnection node_id=0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: connecting to 13.215.254.242:9092 [('13.215.254.242', 9092) IPv4]
<BrokerConnection node_id=0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: Connection complete.
<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connected> [IPv4 ('13.215.254.242', 9092)]>: Closing connection. 


{'id': 67833, 'name': 'David Nelson', 'address': '979 Deborah Tunnel\nFryeland, HI 19406', 'amount': 768480}
{'id': 9792, 'name': 'Kathryn Cole', 'address': '1950 Anna Gardens\nMeganfurt, GA 46532', 'amount': 765697}
{'id': 38432, 'name': 'Gregory Garcia', 'address': '024 Amy Unions\nSusanview, NH 86239', 'amount': 665433}
{'id': 16873, 'name': 'Morgan Patterson', 'address': '9113 Brad Villages\nNorth Timothyport, NH 11206', 'amount': 890315}
{'id': 78792, 'name': 'Kendra Zimmerman', 'address': '8303 Jennifer Extension Suite 675\nLunahaven, NC 86908', 'amount': 37866}
{'id': 50180, 'name': 'Anthony Rogers', 'address': '6435 James Ports\nJeffreychester, DE 94668', 'amount': 54116}
{'id': 59099, 'name': 'Christine Webster', 'address': '3150 Brown Vista Apt. 801\nCourtneyborough, KS 11777', 'amount': 405119}
{'id': 32972, 'name': 'Brenda Flores', 'address': '64859 Danielle Viaduct Suite 091\nRileyfort, KS 41645', 'amount': 908732}
{'id': 94368, 'name': 'Laura Brewer', 'address': '5496 Ray

In [7]:
# this will keep track of records read and will ignore them in re-reading data
# Do NOT run this cell unless you want to go back to beginning for ingestion
consumer_offset = -1

In [8]:
#list of records received
fullrecords=[]
datarecords=[]

In [20]:
# CONSUMER
print("\n**Starting Consumer**")
consumer = KafkaConsumer(bootstrap_servers=[server],
                         auto_offset_reset='earliest',
#                        security_protocol='SASL_PLAINTEXT',
#                        sasl_mechanism='PLAIN',
#                        sasl_plain_username='mapr',
#                        sasl_plain_password='mapr'
                        )

consumer.subscribe(topic)
numMsgConsumed = 0

for _ in range(10):
    records = consumer.poll(timeout_ms=500)
    for topic_data, consumer_records in records.items():
        for consumer_record in consumer_records:
            if(consumer_record.offset > consumer_offset):
                print("Offset: " + str(consumer_record.offset) + "  Received message: " + consumer_record.value.decode('utf-8'))
                fullrecords.append(consumer_record)
                datarecords.append(json.loads(consumer_record.value.decode('utf-8')))
                consumer_offset = consumer_record.offset
                numMsgConsumed += 1
            else:
                print("Offset: " + str(consumer_record.offset) + " Already consumed")
print("New Messages consumed: " + str(numMsgConsumed))

<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: connecting to 13.215.254.242:9092 [('13.215.254.242', 9092) IPv4]
Probing node bootstrap-0 broker version
<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: Connection complete.
Broker version identified as 2.5.0
Set configuration api_version=(2, 5, 0) to skip auto check_version requests on startup
group_id is None: disabling auto-commit.
Updating subscribed topics to: ['freshtopic']
Updated partition assignment: [TopicPartition(topic='freshtopic', partition=0)]
<BrokerConnection node_id=0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: connecting to 13.215.254.242:9092 [('13.215.254.242', 9092) IPv4]
<BrokerConnection node_id=0 host=13.215.254.242:9092 <connecting> [IPv4 ('13.215.254.242', 9092)]>: Connection complete.
<BrokerConnection node_id=bootstrap-0 host=13.215.254.242:9092 <connected> [I


**Starting Consumer**
Offset: 0 Already consumed
Offset: 1 Already consumed
Offset: 2 Already consumed
Offset: 3 Already consumed
Offset: 4 Already consumed
Offset: 5 Already consumed
Offset: 6 Already consumed
Offset: 7 Already consumed
Offset: 8 Already consumed
Offset: 9 Already consumed
Offset: 10 Already consumed
Offset: 11 Already consumed
Offset: 12 Already consumed
Offset: 13 Already consumed
Offset: 14 Already consumed
Offset: 15 Already consumed
Offset: 16 Already consumed
Offset: 17 Already consumed
Offset: 18 Already consumed
Offset: 19 Already consumed
Offset: 20 Already consumed
Offset: 21 Already consumed
Offset: 22 Already consumed
Offset: 23 Already consumed
Offset: 24 Already consumed
Offset: 25 Already consumed
Offset: 26 Already consumed
Offset: 27 Already consumed
Offset: 28 Already consumed
Offset: 29 Already consumed
Offset: 30 Already consumed
Offset: 31 Already consumed
Offset: 32 Already consumed
Offset: 33 Already consumed
Offset: 34 Already consumed
Offset:

In [21]:
# check offset is tracked correctly
consumer_offset

199

In [22]:
#build pandas data frame from kafka consumer ingested records
optable = pa.Table.from_pandas(pd.DataFrame(datarecords))

In [23]:
uniquestr=tm.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [24]:
pq.write_table(optable,"/mnt/shared/ingest1/source1/kafkaingest1"+uniquestr+".parquet")